In [1]:
import pandas as pd
import numpy as np
import researchpy as rp
from operator import itemgetter

from sklearn.utils import compute_sample_weight
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score, accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

In [18]:
import pickle5 as pickle

file_path = './T5/df_dc_embeded_by_line_small.pkl'
with open(file_path, "rb") as fh:
  df = pickle.load(fh)

In [19]:
df.head()

,sample_id,severity,method,label,embeded_sequence_sum,embeded_sequence_avg
0,4432196,major,@JsonSerialize(include = JsonSerialize.Inclusi...,1,"[10.53414, -6.7710633, -15.971554, 0.46057856,...","[0.22900304, -0.14719702, -0.3472077, 0.010012..."
1,7391055,major,@Value\npublic class Release {\n\n\tprivate fi...,1,"[1.1591874, 0.5549224, -2.3538694, 0.64975923,...","[0.14489843, 0.0693653, -0.29423368, 0.0812199..."
2,4687786,major,public class GroomServerStatus implements Writ...,1,"[46.416496, -17.33269, -57.63364, 12.103407, -...","[0.2937753, -0.10970057, -0.36476988, 0.076603..."
3,3797964,major,public static class Value {\n int ...,1,"[1.2814045, -0.7290777, -0.9958315, 0.04294371...","[0.32035112, -0.18226942, -0.24895787, 0.01073..."
4,5339993,major,public class InstantiatedVersionedLabel extend...,1,"[2.3589935, -2.0929816, -5.560404, 1.9338074, ...","[0.1474371, -0.13081135, -0.34752524, 0.120862..."


In [20]:
embeded_sum = df.loc[:, ['sample_id', 'severity', 'label', 'embeded_sequence_sum']]
embeded_avg = df.loc[:, ['sample_id', 'severity', 'label', 'embeded_sequence_avg']]
embeded_sum.head()

,sample_id,severity,label,embeded_sequence_sum
0,4432196,major,1,"[10.53414, -6.7710633, -15.971554, 0.46057856,..."
1,7391055,major,1,"[1.1591874, 0.5549224, -2.3538694, 0.64975923,..."
2,4687786,major,1,"[46.416496, -17.33269, -57.63364, 12.103407, -..."
3,3797964,major,1,"[1.2814045, -0.7290777, -0.9958315, 0.04294371..."
4,5339993,major,1,"[2.3589935, -2.0929816, -5.560404, 1.9338074, ..."


In [21]:
n = len(embeded_avg.embeded_sequence_avg[0])
n

512

In [27]:
columns = [f'em_{i+1}' for i in range(n)]
data = pd.DataFrame(embeded_sum["embeded_sequence_sum"].to_list(), columns=columns)
data['label'] = embeded_sum['label']
data['sample_id'] = embeded_sum['sample_id']
data['severity'] = embeded_sum['severity']
data.head()

,em_1,em_2,em_3,em_4,em_5,em_6,em_7,em_8,em_9,em_10,...,em_506,em_507,em_508,em_509,em_510,em_511,em_512,label,sample_id,severity
0,10.534140,-6.771063,-15.971554,0.460579,-7.727417,-10.533665,-15.832234,9.679967,26.604067,6.078239,...,-20.516022,3.662830,2.023729,-16.105082,3.613373,-0.464404,5.504441,1,4432196,major
1,1.159187,0.554922,-2.353869,0.649759,1.084038,1.240736,-1.778987,2.639224,4.610524,0.498727,...,-4.117848,1.101393,0.661132,-1.724599,1.432518,0.351758,-1.012303,1,7391055,major
2,46.416496,-17.332689,-57.633640,12.103407,-8.553068,-30.315754,-63.244621,39.642513,63.038857,19.866613,...,-34.748493,0.339229,-5.721843,-29.626919,-5.367094,12.793773,11.151268,1,4687786,major
3,1.281404,-0.729078,-0.995831,0.042944,-0.231063,-0.300575,-0.384148,1.328988,1.568126,0.295318,...,-2.269326,-0.776043,0.206616,-2.222340,-0.484990,-0.208199,1.453414,1,3797964,major
4,2.358994,-2.092982,-5.560404,1.933807,-2.516862,-3.112549,-5.154471,2.676773,11.468258,1.369152,...,-7.982023,-0.553051,-0.432792,-3.331370,-0.832675,3.482531,3.756674,1,5339993,major


In [28]:
dataset_path = './embedded_datasets/T5_small_line_sum.pkl'
pd.to_pickle(data, dataset_path)

In [29]:
columns = [f'em_{i+1}' for i in range(n)]
data = pd.DataFrame(embeded_avg["embeded_sequence_avg"].to_list(), columns=columns)
data['label'] = embeded_avg['label']
data['sample_id'] = embeded_avg['sample_id']
data['severity'] = embeded_avg['severity']
data.head()

,em_1,em_2,em_3,em_4,em_5,em_6,em_7,em_8,em_9,em_10,...,em_506,em_507,em_508,em_509,em_510,em_511,em_512,label,sample_id,severity
0,0.229003,-0.147197,-0.347208,0.010013,-0.167987,-0.228993,-0.344179,0.210434,0.578349,0.132136,...,-0.446000,0.079627,0.043994,-0.350110,0.078552,-0.010096,0.119662,1,4432196,major
1,0.144898,0.069365,-0.294234,0.081220,0.135505,0.155092,-0.222373,0.329903,0.576316,0.062341,...,-0.514731,0.137674,0.082642,-0.215575,0.179065,0.043970,-0.126538,1,7391055,major
2,0.293775,-0.109701,-0.364770,0.076604,-0.054133,-0.191872,-0.400282,0.250902,0.398980,0.125738,...,-0.219927,0.002147,-0.036214,-0.187512,-0.033969,0.080973,0.070578,1,4687786,major
3,0.320351,-0.182269,-0.248958,0.010736,-0.057766,-0.075144,-0.096037,0.332247,0.392031,0.073829,...,-0.567332,-0.194011,0.051654,-0.555585,-0.121248,-0.052050,0.363354,1,3797964,major
4,0.147437,-0.130811,-0.347525,0.120863,-0.157304,-0.194534,-0.322154,0.167298,0.716766,0.085572,...,-0.498876,-0.034566,-0.027049,-0.208211,-0.052042,0.217658,0.234792,1,5339993,major


In [30]:
dataset_path = './embedded_datasets/T5_small_line_avg.pkl'
pd.to_pickle(data, dataset_path)

### Class embedding

In [108]:
import pickle5 as pickle

file_path = '../data/T5/df_dc_embeded_small.pkl'
with open(file_path, "rb") as fh:
  df = pickle.load(fh)

print(len(df))
df.head()

2159


,sample_id,severity,method,label,embeded_sequence
0,4432196,major,@JsonSerialize(include = JsonSerialize.Inclusi...,1,"[0.12319732, 0.013010962, -0.36437827, 0.13048..."
1,7391055,major,@Value\npublic class Release {\n\n\tprivate fi...,1,"[0.08438608, -0.08265672, -0.3884728, 0.285299..."
2,4687786,major,public class GroomServerStatus implements Writ...,1,"[0.19245997, -0.045380514, -0.3891685, 0.16218..."
3,3797964,major,public static class Value {\n int ...,1,"[0.048406683, -0.0444143, -0.30889696, 0.21213..."
4,5339993,major,public class InstantiatedVersionedLabel extend...,1,"[0.041008048, 0.04622683, -0.35491765, 0.47511..."


In [109]:
cols_with_null(df)

                  #NaN
embeded_sequence   217


In [110]:
df = df.dropna()
len(df)

1942

In [111]:
n = len(df.embeded_sequence[0])
n

512

In [112]:
columns = [f'em_{i+1}' for i in range(n)]
df[columns] = df["embeded_sequence"].to_list()

Empty DataFrame
Columns: [#NaN]
Index: []


In [113]:
df.drop(columns=['method', 'embeded_sequence'], inplace=True)
df.head()

,sample_id,severity,label,em_1,em_2,em_3,em_4,em_5,em_6,em_7,...,em_503,em_504,em_505,em_506,em_507,em_508,em_509,em_510,em_511,em_512
0,4432196,major,1,0.123197,0.013011,-0.364378,0.130486,0.177271,-0.220527,-0.614902,...,-0.164616,-0.045421,-0.087850,-0.415469,-0.250676,-0.034011,-0.177202,-0.224473,0.215671,-0.043203
1,7391055,major,1,0.084386,-0.082657,-0.388473,0.285300,-0.018974,-0.237432,-0.377954,...,-0.068464,-0.205118,0.054537,-0.435160,0.077640,0.148767,-0.060151,0.061936,0.126259,-0.046671
2,4687786,major,1,0.192460,-0.045381,-0.389169,0.162181,0.123672,-0.235752,-0.544954,...,-0.130225,0.041419,-0.038299,-0.260974,-0.197850,-0.173269,-0.045579,-0.147961,0.099592,-0.015848
3,3797964,major,1,0.048407,-0.044414,-0.308897,0.212130,-0.193294,-0.290775,-0.694648,...,-0.188465,0.357595,-0.094530,-0.515571,-0.320304,0.149350,-0.446925,-0.093794,0.099125,0.127002
4,5339993,major,1,0.041008,0.046227,-0.354918,0.475114,0.116208,-0.284086,-0.555644,...,-0.106989,-0.036921,-0.062708,-0.592264,-0.232006,0.107929,-0.058925,-0.226961,0.302240,0.090362


In [114]:
dataset_path = './embedded_datasets/T5_small.pkl'
pd.to_pickle(df, dataset_path)